# Homework 2
ML-Zoomcamp


In [20]:
# load libraries
import numpy as np
import pandas as pd

#load data
df = pd.read_csv('AB_NYC_2019.csv')

In [21]:
#select only several features
df = df[['latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365']]
df

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64749,-73.97237,149,1,9,0.21,6,365
1,40.75362,-73.98377,225,1,45,0.38,2,355
2,40.80902,-73.94190,150,3,0,NaN,1,365
3,40.68514,-73.95976,89,1,270,4.64,1,194
4,40.79851,-73.94399,80,10,9,0.10,1,0
...,...,...,...,...,...,...,...,...
48890,40.67853,-73.94995,70,2,0,NaN,2,9
48891,40.70184,-73.93317,40,4,0,NaN,2,36
48892,40.81475,-73.94867,115,10,0,NaN,1,27
48893,40.75751,-73.99112,55,1,0,NaN,6,2


## Q1: Find a feature with missing values

In [22]:
df.isnull().sum()

latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

reviews_per_month has 10052 null values

## Q2: Quantile for minimum_nights

In [23]:
df["minimum_nights"].quantile()
#np.median(df["minimum_nights"]) #or median directly since 50% is just median

3.0

## Splitting the data

In [29]:
# shuffle seed 42
idx = np.arange(len(df))
np.random.seed(42)
np.random.shuffle(idx)

#split into 60/20/20
n_train = int(0.6 *  len(df))
n_val = int(0.2 *  len(df))
n_test = len(df) - n_train - n_val

#get unique index
idx_train = idx[:n_train]
idx_val = idx[n_train:n_train+n_val]
idx_test = idx[n_train+n_val:]

#get df
df_train = df.iloc[idx_train]
df_val = df.iloc[idx_val]
df_test = df.iloc[idx_test]


#remove price (output label) 
del df_train["price"]
del df_val["price"]
del df_test["price"]

#get regression values
y_train = np.log1p(df.iloc[idx_train]["price"].values)
y_val = np.log1p(df.iloc[idx_val]["price"].values)
y_test = np.log1p(df.iloc[idx_test]["price"].values)

## Q3: zero vs mean

In [76]:
# Should we use zeros or mean? 

#filling it with 0
df0 = df.fillna(0)

#filling with mean
dfm = df.fillna(df.mean())

In [110]:
#train linear regression function
def train_linear_regression(X,y, r = 0):
   
    # get Numpy array x^0 + x^1
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones,X])
    
    # X'X
    XTX = X.T.dot(X) + r * np.eye(X.shape[1])
    XTXinv = np.linalg.inv(XTX)
    w = XTXinv.dot(X.T).dot(y)
    return w


def linear_regression(X,w):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones,X])
    a= np.dot(X,w)
    return a

def rmse(x,y):
    return np.sqrt(np.mean((x-y)**2))


In [111]:
# results of training by filling NaN with zeros
df0_train = df0.iloc[idx_train]
del df0_train["price"]

X0_train = df0_train.values
w0 = train_linear_regression(X0_train,y_train)

y0_val = linear_regression(df_val.fillna(0).values,w0)
#y0_train
print("Using Fill NA with 0, RMSE is %2.2f" % round(rmse(y0_val,y_val),2))


Using Fill NA with 0, RMSE is 0.64


In [112]:
# results of training by filling NaN with mean
dfm_train = dfm.iloc[idx_train]
del dfm_train["price"]

Xm_train = dfm_train.values
wm = train_linear_regression(Xm_train,y_train)

ym_val = linear_regression(df_val.fillna(df_val.mean()).values,w0)
#y0_train
print("Using Fill NA with mean, RMSE is %2.2f" % round(rmse(ym_val,y_val),2))


Using Fill NA with mean, RMSE is 0.64


In [114]:
#--------- extra-----
#hmmm, this doesn't make sense, was the error this small?

print("L2 norm error for NA -> 0: %2.8f" % rmse(y0_val,y_val))
print("L2 norm error for NA -> Mean %2.8f" % rmse(ym_val,y_val))

#ah because we are changing NA in both of outputs and inputs?

L2 norm error for NA -> 0: 0.64303378
L2 norm error for NA -> Mean 0.64328657


## Q4: Little bit of Regularization

In [117]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    X0_train = df0_train.values
    w0 = train_linear_regression(X0_train,y_train,r)
    y0_val = linear_regression(df_val.fillna(0).values,w0)

    print("%2.5f \t %2.2f" % ( r, round(rmse(y0_val,y_val),2)))

0.00000 	 0.64
0.00000 	 0.64
0.00010 	 0.64
0.00100 	 0.64
0.01000 	 0.66
0.10000 	 0.68
1.00000 	 0.68
5.00000 	 0.68
10.00000 	 0.68


## Q5: std

In [130]:
allv = [];
for seed in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    idx = np.arange(len(df))
    np.random.seed(seed)
    np.random.shuffle(idx)

    #split into 60/20/20
    n_train = int(0.6 *  len(df))
    n_val = int(0.2 *  len(df))
    n_test = len(df) - n_train - n_val

    #get unique index
    idx_train = idx[:n_train]
    idx_val = idx[n_train:n_train+n_val]
    idx_test = idx[n_train+n_val:]

    #get df
    df_train = df.iloc[idx_train].fillna(0)
    df_val = df.iloc[idx_val].fillna(0)
    df_test = df.iloc[idx_test].fillna(0)


    #remove price (output label) 
    del df_train["price"]
    del df_val["price"]
    del df_test["price"]

    #get regression values
    y_train = np.log1p(df.iloc[idx_train]["price"].values)
    y_val = np.log1p(df.iloc[idx_val]["price"].values)
    y_test = np.log1p(df.iloc[idx_test]["price"].values)


    X_train = df_train.values
    w = train_linear_regression(X_train,y_train,r=0)
    yhat_val = linear_regression(df_val.fillna(0).values,w)
    print("seed: %d \t rmse:%2.2f" % ( seed, round(rmse(yhat_val,y_val),2)))

    allv.append(rmse(yhat_val,y_val))

stdrmse  = np.std(allv)
print("std of RMSE: %.3f" % round(stdrmse,3))

seed: 0 	 rmse:0.65
seed: 1 	 rmse:0.65
seed: 2 	 rmse:0.65
seed: 3 	 rmse:0.64
seed: 4 	 rmse:0.64
seed: 5 	 rmse:0.63
seed: 6 	 rmse:0.63
seed: 7 	 rmse:0.65
seed: 8 	 rmse:0.65
seed: 9 	 rmse:0.64
std of RMSE: 0.008


## Q6: RMSE on test datatest

In [129]:
seed = 9;
idx = np.arange(len(df))
np.random.seed(seed)
np.random.shuffle(idx)

#split into 80/20
n_trainval = int(0.8 *  len(df))
n_test = len(df) - n_trainval

#get unique index
idx_trainval = idx[:n_trainval]
idx_test = idx[n_trainval:]

#get df
df_trainval = df.iloc[idx_trainval].fillna(0)
df_test = df.iloc[idx_test].fillna(0)


#remove price (output label) 
del df_trainval["price"]
del df_test["price"]

#get regression values
y_trainval = np.log1p(df.iloc[idx_trainval]["price"].values)
y_test = np.log1p(df.iloc[idx_test]["price"].values)


X_trainval = df_trainval.values
w = train_linear_regression(X_trainval,y_trainval,r=0.001)
yhat_test = linear_regression(df_test.fillna(0).values,w)
print("seed: %d \t rmse:%2.2f" % ( seed, round(rmse(yhat_test,y_test),2)))


seed: 9 	 rmse:0.65


Completed! :D